In [1]:
from utils import (
    get_areas, 
    get_specimen_ids, 
    get_trials,
    get_units_count,
    get_summary_df,
    load_index
)

from mapping import (
    consistency_across_trials,
    interanimal_consistency_1v1,
    interanimal_consistency_pool
)

In [2]:
index_df = load_index("../Preproc2/data/combined_index.csv")

# Theoretical background

## General framework

The mouse representation after each stimulus (i.e. image) are stored in tensors of shape $(T, I, U)$, where:
- $T$ is the number of trials (i.e. repetitions of the same stimulus)
- $I$ is the number of images (i.e. different stimuli)
- $U$ is the number of units (i.e. neurons) in a pairt (specimen, area)


We want to compute the similarity between two representations, for example same (specimen, area) and different trials (fixed stimulus), or same area, different specimens (fixed stimulus), or the similarity between a model representation (e.g. one layer of AlexNet) and a (specimen, area) representation (fixed stimulus).


## Corrected similarity

In all cases we have to deal with very noisy signals, so we will use a statistical methods to estimate the **corrected similarity**, exploiting the repetitions of the same stimulus (trials).

Let $A$ and $B$ be two representations of shape $(T, I, U_A)$ and $(T, I, U_B)$ respectively (e.g. $A$ could be a model representation, $B$ a neural representation, i.e. a specific specimen and area representation).

$$
\text{sim}_\text{corrected}(A,B) = \frac{\text{sim}_\text{observed}(A,B)}{\sqrt{\text{reliability}(A) \cdot \text{reliability}(B)}}
$$

One way to estimate the reliability of a representation is to compute the similarity between two halves of the trials, for example using split-half correlation (Spearman-Brown corrected):

$$
\text{reliability}(A) = \text{SB}(\text{sim}_\text{observed}(A_1, A_2))
$$
$$
\text{SB}(r) = \frac{2r}{1+r}
$$

where $A_1$ and $A_2$ are defined by taking two halves of the trials of representation $A$, and computing the mean response over trials for each half. So $A_1$ and $A_2$ have shape $(I, U_A)$, i.e. they are the traditional **design matrices**.

The split is done by randomly assigning each trial to one of the two halves, and then averaging the similarity over multiple random splits (n_bootstrap=100).


### RSA, CKA similarity

$$
\text{RSA}_{\text{corrected}}(A,B) = \frac{\frac{1}{2}[\text{RSA}_{\text{observed}}(A_1,B_2) + \text{RSA}_{\text{observed}}(A_2,B_1)]}{\sqrt{\text{reliability}(A) \cdot \text{reliability}(B)}}
$$

analogously for CKA.

### PLS regression

Given two representations $A$ and $B$, we fit a PLS regression model to predict $B$ from $A$ (with 25 components, as in the paper), and then we compute the corrected correlation between the predicted $\hat{B}$ and the true $B$.

We performed a train-test split on the images (50% train, 50% test): $A_{train}, A_{test}, B_{train}, B_{test}$.

We split each of the train and test sets in two halves of trials: $A_{train,1}, A_{train,2}, A_{test,1}, A_{test,2}, B_{train,1}, B_{train,2}, B_{test,1}, B_{test,2}$.

We fit two PLS models: one on the first half of trials, one on the second half of trials: 

$$
f_1: A_{train,1} \rightarrow B_{train,1}
$$

$$
f_2: A_{train,2} \rightarrow B_{train,2}
$$

We use the two models to predict the two halves of the test set:

$$
\hat{B}_1 = f_1(A_{test,1})
$$

$$
\hat{B}_2 = f_2(A_{test,2})
$$

Then for each unit $j$ in $B$, we compute the corrected correlation between the predicted and true responses:

$$
s_j = \frac{\text{corr}(\hat{B}_1^j, B_2^j)}{\sqrt{\text{SB}(\text{corr}(\hat{B}_1^j,\hat{B}_2^j)) \cdot \text{SB}(\text{corr}(B_1^j,B_2^j))}}

$$

Finally we take the median over all units:

$$
\text{PLS}_{\text{corrected}}(A,B) = \text{median}_j(s_j)
$$

## Comparing a mouse with itself across different areas

Using the above formalism, we compute here the similarity between the representations each pair (specimen, area) and itself. This is just a trivial check to ensure that the code is working as expected, i.e. the similarity should be 1 (or very close to 1).

In [3]:
consistency_rsa = consistency_across_trials(index_df, sim_metric='RSA')
consistency_rsa

,Area,Specimen ID,Mean,Std
0,VISal,735109609,1.000424,0.010890
1,VISal,738651054,1.001548,0.011861
2,VISal,739783171,1.000526,0.008132
3,VISal,742714475,0.999715,0.008578
4,VISal,757329624,1.002928,0.014106
5,VISal,775876828,1.006321,0.020463
6,VISam,730756780,1.005835,0.015772
7,VISam,734865738,1.016318,0.024636
8,VISam,738651054,1.003464,0.016800
9,VISam,739783171,1.005976,0.017088


In [4]:
# consistency_cka = consistency_across_trials(index_df, sim_metric='CKA')
# consistency_cka

In [ ]:
consistency_PLS = consistency_across_trials(index_df, sim_metric='PLS')
consistency_PLS

# 1 vs 1 mapping: interanimal consistency

Now we can compute the similarity between each pair ofdifferent specimens, for the same area. This is the inter-animal consistency, as defined in the paper. Then we aggregate the results by taking the median across all pairs of specimens, for each area.

In [6]:
interanimal_consistency_1v1_df_rsa = interanimal_consistency_1v1(index_df, sim_metric='RSA')
interanimal_consistency_1v1_df_rsa.groupby('Area')['Mean'].mean() #median()

Area
VISal    0.444850
VISam    0.312528
VISl     0.174234
VISp     0.331657
VISpm    0.329402
VISrl    0.260001
Name: Mean, dtype: float64

In [7]:
# interanimal_consistency_1v1_df_cka = interanimal_consistency_1v1(index_df, sim_metric='CKA')
# interanimal_consistency_1v1_df_cka.groupby('Area')['Mean'].mean() #median()

In [5]:
interanimal_consistency_1v1_df_pls = interanimal_consistency_1v1(index_df, sim_metric='PLS', n_boot=5, n_splits=5)
interanimal_consistency_1v1_df_pls.groupby('Area')['Mean'].mean() #median()

/home/spina/Desktop/units/DL/project/mice-representation/.mouse_vision/lib/python3.13/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/spina/Desktop/units/DL/project/mice-representation/.mouse_vision/lib/python3.13/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/spina/Desktop/units/DL/project/mice-representation/.mouse_vision/lib/python3.13/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/spina/Desktop/units/DL/project/mice-representation/.mouse_vision/lib/python3.13/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of

Area
VISal    0.393975
VISam    0.328946
VISl     0.199392
VISp     0.295685
VISpm    0.318515
VISrl    0.295475
Name: Mean, dtype: float64

## Pooling version

In the pooling version, we compute the similarity between each specimen and the pooled representation of all the other specimens, for the same area. Then we aggregate the results by taking the median across all specimens, for each area.

In [3]:
iac_pool_rsa = interanimal_consistency_pool(index_df, sim_metric='RSA')
iac_pool_rsa.groupby('Area')['Mean'].mean() #median()

Area
VISal    0.598281
VISam    0.479374
VISl     0.300002
VISp     0.499127
VISpm    0.465677
VISrl    0.427804
Name: Mean, dtype: float64

In [ ]:
# iac_pool_cka = interanimal_consistency_pool(index_df, sim_metric='CKA')
# iac_pool_cka.groupby('Area')['Mean'].mean() #median()

In [4]:
iac_pool_pls = interanimal_consistency_pool(index_df, sim_metric='PLS', n_boot=5, n_splits=5)
iac_pool_pls.groupby('Area')['Mean'].mean() #median()

/home/spina/Desktop/units/DL/project/mice-representation/.mouse_vision/lib/python3.13/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/spina/Desktop/units/DL/project/mice-representation/.mouse_vision/lib/python3.13/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/spina/Desktop/units/DL/project/mice-representation/.mouse_vision/lib/python3.13/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of iterations reached
  warnings.warn("Maximum number of iterations reached", ConvergenceWarning)
/home/spina/Desktop/units/DL/project/mice-representation/.mouse_vision/lib/python3.13/site-packages/sklearn/cross_decomposition/_pls.py:104: ConvergenceWarning: Maximum number of

Area
VISal    0.541422
VISam    0.484949
VISl     0.333043
VISp     0.457581
VISpm    0.462249
VISrl    0.461443
Name: Mean, dtype: float64